In [97]:
import pandas as pd
import numpy as np
path = "ECAI"

def get_intersection(v01, ve1, v02, ve2):
    # handle division by zero masking out the positions where the denominator is zero
    denominator = ve2 - ve1
    if isinstance(denominator, np.ndarray):
        denominator[denominator == 0] = np.nan
    else:
        if denominator == 0:
            return np.nan
    wes = (v01 - v02) / denominator
    return np.round(wes[(wes >= 0) & (wes <= 10)], 2)

In [98]:
# New method
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"
unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2)

# wes  = get_intersection(ethical_np[:, 0, 0], ethical_np[:, 0, 1], unethical_np[:, 0, 1], unethical_np[:, 0, 1])
wes = get_intersection(unethical_np[:, 0, 0], unethical_np[:, 0, 1], ethical_np[:, 0, 0], ethical_np[:, 0, 1])
wes.mean(), wes.std(), np.median(wes)

(2.4503655660377355, 1.7335857520635367, 2.15)

In [99]:
# Old method
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"
unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2).mean(axis=0)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2).mean(axis=0)

wes  = get_intersection(unethical_np[0, 0], unethical_np[0, 1], ethical_np[0, 0], ethical_np[0, 1])
wes.mean(), wes.std()

(1.81, 0.0)